# GETTING THE DATA READY 
I have used excel to manipulate the data into a valuable set theta i would use. Using the FactInternetSales datasets I broke it down into a suitable sustainable form.

###  CustomerKey
The customer key gives us a leeway into the DimCustomer table. from here we get 29 columns but not all will be of interest for this aanlysis. we used the VLOOKUP function in excel to get the exact geography key for every custommer based on the customerkey. The CustomerAlternateKey	Title	FirstName	MiddleName	LastName	NameStyle columns re not that necesary for this analysis incomparison to the content they store. we have an interest in sales  and everything feasible about the sales and customers. The birthdate will be important in geting the customers age rane and understand the buying patterns of different agegroups. Gender is also important in understanding what goods are common among diferent genders. Email adress are not necesary for analysis. YearlyIncome	TotalChildren	NumberChildrenAtHome are all important to understand customers buying the specific goods and know how the number of children will affect the buuying patterns and what they will buy.EnglishEducation will help us understanding the buying behavour based on the level of education. HouseOwnerFlag	NumberCarsOwned CommuteDistance are also eseantial in this analysis as they will prob further understandingt he buying pattersn based on cars owned and if the disytance has a significant affect on buying behavior. from the DimCustomer table we  will only pick thee mentioned tables.

###  ProductKey
the Product key gives as an insight into the dimProduct data. from this data we only get the actual products name so that we can have an exact product names instead of product keys. in this we also use the VLOOKUP function to get the ecact key along with its name onto the FactinternetSales data.

### DimProductSubcategoryKey
This gives us the leeway to  DimProductSubcategory dataset where we get the subcategory names of each datasset into our datset.

###   DimSalesTerritoryKey

DimSalesTerritoryKey allows for the matching of data in the DimSalesTerritory data and get the Teritory names for each teritory location.

###     geography key
this key points us to DimGeography where we  get cityy country and locations in general of the customers.

###   CurrencyKey

DimCurrency is directe related to this key where we get curency name used instead of  having the curency keys.

     -  **these are the datasets chosen with their respective columns. these tables are merged into the FactinternetSales data table**

###   SalesTerritoryKey
this column helps in getting the sales by teritory for the sakes agenst and also get the targets set fro them in the  FactSalesTargets dataset. 

###   TABLES NOT CHOSEM
    - DimAccount helps in understanding the account types for customers which is not a point of interest in this analysis. 
    - DimDate is dates stored with weekid and names of the weeks which can still be imported from python without the need of the table being added into the dataset.
    - DimDepartmentGroup  is the departmental grouping of thedepartments in the  DimOrganization ddata.
    - DimPromotion DimReseller DimSalesReason DimScenario FactCallCenter FactCurrencyRate FactFinance 

Examining the summary statistics for the selected columns from the merged 'Geography' and 'DimCustomer' tables, I find insightful details about various customer attributes. Beginning with 'YearlyIncome,' the dataset encompasses 18,484 entries, reflecting a diverse range of annual incomes. The average yearly income stands at approximately $57,305.78, with a notable standard deviation of $32,285.84, indicating considerable variability in income levels.

Moving on to family-related attributes, 'TotalChildren' showcases an average of 1.84 children per customer, while 'NumberChildrenAtHome' indicates an average of about 1.00 child residing at home. Both metrics exhibit some degree of variability, with customers having up to 5 children. 'HouseOwnerFlag' reveals that, on average, about 68% of customers own a house, with a minimum value of 0 indicating non-homeownership for some individuals.

Considering lifestyle indicators, 'NumberCarsOwned' illustrates that customers own an average of 1.50 cars, with some not owning any and others having up to 4 cars. Lastly, 'SalesTerritoryKey' provides insights into the distribution of sales territory keys, with an average of approximately 5.90. These metrics, each offering unique perspectives on customer characteristics, collectively contribute to a comprehensive understanding of the merged dataset, excluding 'GeographyKey' and 'CustomerKey' columns.

In [1]:
from pyspark.sql import SparkSession
import os

# Initialize a Spark session
spark = SparkSession.builder.appName("CSVLoader").getOrCreate()

# Path to the folder containing CSV files
folder_path = r'C:\Users\neste\OneDrive\Desktop\karanja\DataSet_final\DataSet_final'

# Get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Create variables for each DataFrame
for csv_file in csv_files:
    # Use the file name (without extension) as the variable name
    df_name = os.path.splitext(csv_file)[0]
    # Read the CSV file into a DataFrame
    globals()[df_name] = spark.read.csv(os.path.join(folder_path, csv_file), header=True, inferSchema=True)

# Show the contents of each DataFrame
DimGeography.show()
DimAccount.show()
DimCurrency.show()
DimCustomer.show()
DimDate.show()
DimDepartmentGroup.show()
DimOrganization.show()
DimProduct.show()
DimProductCategory.show()
DimProductSubcategory.show()
DimPromotion.show()
DimReseller.show()
DimSalesReason.show()
DimSalesTerritory.show()
DimScenario.show()
FactCallCenter.show()
FactCurrencyRate.show()
FactInternetSales.show()



+------------+--------------+-----------------+-----------------+-----------------+------------------------+------------------------+-----------------------+----------+-----------------+----------------+
|GeographyKey|          City|StateProvinceCode|StateProvinceName|CountryRegionCode|EnglishCountryRegionName|SpanishCountryRegionName|FrenchCountryRegionName|PostalCode|SalesTerritoryKey|IpAddressLocator|
+------------+--------------+-----------------+-----------------+-----------------+------------------------+------------------------+-----------------------+----------+-----------------+----------------+
|           1|    Alexandria|              NSW|  New South Wales|               AU|               Australia|               Australia|              Australie|      2015|                9|    198.51.100.2|
|           2| Coffs Harbour|              NSW|  New South Wales|               AU|               Australia|               Australia|              Australie|      2450|                

# data cleaning 

here we merge the dataset into a suitable one first we get product name from the dimproduct table into the factinternetsales

from dimCustomer we get the followin columns uning the CustomerKey. GeographyKey | BirthDate MaritalStatus Gender TotalChildren NumberChildrenAtHome   EnglishEducation HouseOwnerFlag NumberCarsOwned

fromthe DimPromotion we get the EnglishPromotionName into the df.

from the dimCurency we get the CurrencyKey column whhich helps us to get the Curency names from the dimCurency table.

from dim salesTertoy we use the saleyterirorykey to join SalesTerritoryRegion	SalesTerritoryCountry	SalesTerritoryGroup columns to the df. thee following columns are droped ["GeographyKey", "SalesTerritoryKey", "CurrencyKey", "PromotionKey", "CustomerKey", "ProductKey", "OrderDateKey", "DueDateKey", "ShipDateKey", "SalesOrderNumber", "CarrierTrackingNumber", "CustomerPONumber", "OrderDate", "DueDate", "ShipDate"]

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, substring
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime, unix_timestamp, date_format, year, month, dayofweek
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime, unix_timestamp, year, month, dayofweek, datediff, lit

# Select only the necessary columns from DimProduct
DimProductSubset = DimProduct.select("ProductKey", "EnglishProductName")

# Perform the join based on the ProductKey column
joined_df = FactInternetSales.join(DimProductSubset, "ProductKey", "inner")


# Select only the necessary columns from DimCustomer
DimCustomerSubset = DimCustomer.select("CustomerKey", "GeographyKey", "BirthDate", "MaritalStatus",
                                       "Gender", "TotalChildren", "NumberChildrenAtHome","CommuteDistance",
                                       "EnglishEducation", "HouseOwnerFlag", "NumberCarsOwned")

# Perform the join based on the CustomerKey column and overwrite joined_df
joined_df = joined_df.join(DimCustomerSubset, "CustomerKey", "inner")
# Select only the necessary columns from DimPromotion
DimPromotionSubset = DimPromotion.select("PromotionKey", "EnglishPromotionName")

# Perform the join with DimPromotion based on the PromotionKey column
joined_df = joined_df.join(DimPromotionSubset, "PromotionKey", "inner")

# Select only the necessary columns from DimCurrency
DimCurrencySubset = DimCurrency.select("CurrencyKey", "CurrencyName")

# Perform the join with DimCurrency based on the CurrencyKey column
joined_df = joined_df.join(DimCurrencySubset, "CurrencyKey", "inner")


# Select only the necessary columns from DimSalesTerritory
DimSalesTerritorySubset = DimSalesTerritory.select("SalesTerritoryKey", "SalesTerritoryRegion", 
                                                   "SalesTerritoryCountry", "SalesTerritoryGroup")

# Perform the join with DimSalesTerritory based on the SalesTerritoryKey column
joined_df = joined_df.join(DimSalesTerritorySubset, "SalesTerritoryKey", "inner")
# Select only the necessary columns from DimGeography
DimGeographySubset = DimGeography.select("GeographyKey", "StateProvinceName", 
                                         "EnglishCountryRegionName", "City")

# Perform the join with DimGeography based on the GeographyKey column
joined_df = joined_df.join(DimGeographySubset, "GeographyKey", "inner")

# Add a "/" after the first four digits and the next two digits in OrderDateKey
joined_df = joined_df.withColumn("OrderDateKey", 
                                 concat_ws("/", 
                                            substring("OrderDateKey", 1, 4),
                                            substring("OrderDateKey", 5, 2),
                                            substring("OrderDateKey", 7, 2)))

# Convert OrderDateKey to a timestamp column
joined_df = joined_df.withColumn("OrderDate", 
                                 from_unixtime(unix_timestamp(joined_df["OrderDateKey"], "yyyy/MM/dd")))

# Extract year, month, and day of week from OrderDate
joined_df = joined_df.withColumn("OrderYear", year("OrderDate"))
joined_df = joined_df.withColumn("OrderMonth", month("OrderDate"))
joined_df = joined_df.withColumn("DayOfWeek", dayofweek("OrderDate"))

# Convert BirthDate to a timestamp column
joined_df = joined_df.withColumn("BirthDate", 
                                 from_unixtime(unix_timestamp(joined_df["BirthDate"], "yyyy-MM-dd")))

# Extract year, month, and day of week from BirthDate
joined_df = joined_df.withColumn("BirthYear", year("BirthDate"))
joined_df = joined_df.withColumn("BirthMonth", month("BirthDate"))
joined_df = joined_df.withColumn("BirthDayOfWeek", dayofweek("BirthDate"))

# Calculate age at the year 2014
current_year = 2014
joined_df = joined_df.withColumn("Age", lit(current_year) - year("BirthDate"))

# Drop the original BirthDate column if needed
joined_df = joined_df.drop("BirthDate")


# Drop specified columns
columns_to_drop = ["GeographyKey", "SalesTerritoryKey", "CurrencyKey", "PromotionKey", 
                    "CustomerKey", "ProductKey", "OrderDateKey", "DueDateKey", "ShipDateKey", 
                    "SalesOrderNumber", "CarrierTrackingNumber", "CustomerPONumber", 
                    "OrderDate", "DueDate", "ShipDate","SalesOrderLineNumber","RevisionNumber","OrderQuantity"]

joined_df = joined_df.drop(*columns_to_drop)

# Show the result
joined_df.show()

+---------+--------------+--------------------+--------------+-------------------+----------------+-----------+--------+-------+--------------------+-------------+------+-------------+--------------------+---------------+-------------------+--------------+---------------+--------------------+--------------------+--------------------+---------------------+-------------------+-------------------+------------------------+-------------+---------+----------+---------+---------+----------+--------------+----+
|UnitPrice|ExtendedAmount|UnitPriceDiscountPct|DiscountAmount|ProductStandardCost|TotalProductCost|SalesAmount|  TaxAmt|Freight|  EnglishProductName|MaritalStatus|Gender|TotalChildren|NumberChildrenAtHome|CommuteDistance|   EnglishEducation|HouseOwnerFlag|NumberCarsOwned|EnglishPromotionName|        CurrencyName|SalesTerritoryRegion|SalesTerritoryCountry|SalesTerritoryGroup|  StateProvinceName|EnglishCountryRegionName|         City|OrderYear|OrderMonth|DayOfWeek|BirthYear|BirthMonth|Bir

## mapping the data
here we mapp th e categorical value to integers mapping them to numericla values

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType

# Specify the categorical columns to convert
categorical_columns = ["EnglishProductName", "MaritalStatus", "Gender", "CommuteDistance", 
                        "EnglishEducation", "EnglishPromotionName", "CurrencyName", 
                        "SalesTerritoryRegion", "SalesTerritoryCountry", "SalesTerritoryGroup", 
                        "StateProvinceName", "EnglishCountryRegionName", "City"]

# Create a dictionary to store the mapping of index to original values for each column
index_mapping = {}

for col_name in categorical_columns:
    # Create a StringIndexer for each categorical column
    indexer = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_index", handleInvalid="keep")
    
    # Fit the indexer and transform the data
    indexed_data = indexer.fit(joined_df).transform(joined_df)
    
    # Collect unique values and their corresponding index
    index_mapping[col_name] = indexed_data.select(f"{col_name}", f"{col_name}_index").distinct()

# Show the mapping of index to original values
for col_name, mapping_df in index_mapping.items():
    print(f"Mapping for {col_name}:")
    mapping_df.show(truncate=False)
    



Mapping for EnglishProductName:
+--------------------------+------------------------+
|EnglishProductName        |EnglishProductName_index|
+--------------------------+------------------------+
|Long-Sleeve Logo Jersey, L|29.0                    |
|Road-250 Red, 58          |48.0                    |
|Mountain-500 Silver, 48   |113.0                   |
|Touring-1000 Blue, 60     |79.0                    |
|Touring-1000 Yellow, 60   |81.0                    |
|Mountain-500 Black, 48    |109.0                   |
|Touring-1000 Blue, 54     |74.0                    |
|Women's Mountain Shorts, M|42.0                    |
|Mountain-100 Silver, 38   |105.0                   |
|Road-550-W Yellow, 42     |49.0                    |
|Mountain-500 Black, 40    |118.0                   |
|Mountain-400-W Silver, 40 |85.0                    |
|Sport-100 Helmet, Red     |4.0                     |
|Women's Mountain Shorts, S|50.0                    |
|Road-550-W Yellow, 40     |61.0                  

In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# Specify the categorical columns to convert
categorical_columns = ["EnglishProductName", "MaritalStatus", "Gender", "CommuteDistance", 
                        "EnglishEducation", "EnglishPromotionName", "CurrencyName", 
                        "SalesTerritoryRegion", "SalesTerritoryCountry", "SalesTerritoryGroup", 
                        "StateProvinceName", "EnglishCountryRegionName", "City"]

# Identify non-categorical columns
non_categorical_columns = [col for col in joined_df.columns if col not in categorical_columns]

# Create a StringIndexer for each categorical column
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_index", handleInvalid="keep") for col in categorical_columns]

# Create a pipeline to apply the StringIndexers
pipeline = Pipeline(stages=indexers)

# Fit and transform the pipeline
joined_df_transformed = pipeline.fit(joined_df).transform(joined_df)

# Select non-categorical columns and the indexed columns
selected_columns = non_categorical_columns + [f"{col}_index" for col in categorical_columns]
joined_df_transformed = joined_df_transformed.select(selected_columns)

# Show the result
joined_df_transformed.show()


+---------+--------------+--------------------+--------------+-------------------+----------------+-----------+--------+-------+-------------+--------------------+--------------+---------------+---------+----------+---------+---------+----------+--------------+----+------------------------+-------------------+------------+---------------------+----------------------+--------------------------+------------------+--------------------------+---------------------------+-------------------------+-----------------------+------------------------------+----------+
|UnitPrice|ExtendedAmount|UnitPriceDiscountPct|DiscountAmount|ProductStandardCost|TotalProductCost|SalesAmount|  TaxAmt|Freight|TotalChildren|NumberChildrenAtHome|HouseOwnerFlag|NumberCarsOwned|OrderYear|OrderMonth|DayOfWeek|BirthYear|BirthMonth|BirthDayOfWeek| Age|EnglishProductName_index|MaritalStatus_index|Gender_index|CommuteDistance_index|EnglishEducation_index|EnglishPromotionName_index|CurrencyName_index|SalesTerritoryRegion_i

# DESCREPTIVE STATISTICS

1. **SalesOrderLineNumber:**
   - This column represents the line number of each sales order.
   - In real life, it's crucial for tracking and organizing orders. Higher line numbers may indicate more items or a more complex order.

2. **RevisionNumber:**
   - All values are 1, indicating no revisions to the orders.
   - In practical terms, this suggests that the sales orders underwent no changes or modifications after initial creation.

3. **OrderQuantity:**
   - The average order quantity is 1, with a standard deviation of 0.
   - This implies that most orders consist of a single item, but some may have more. Understanding average quantities helps in inventory planning.

4. **UnitPrice:**
   - The average unit price is $486.09, with a considerable standard deviation.
   - This suggests a diverse range of products with varying prices. High standard deviation may indicate the presence of both low and high-cost items.

5. **ExtendedAmount:**
   - This column mirrors the unit price values, indicating the total amount for each order.
   - Businesses can analyze this to understand revenue patterns and identify high-value transactions.

6. **UnitPriceDiscountPct:**
   - All values are 0, indicating no unit price discounts.
   - In a real-world scenario, this could mean that the company does not typically offer discounts on unit prices.

7. **DiscountAmount:**
   - All values are 0, indicating no discounts applied.
   - Businesses may use this information to assess the effectiveness of discount strategies.

8. **ProductStandardCost:**
   - The average product standard cost is $286.07.
   - This cost data is essential for calculating profit margins, helping businesses optimize pricing strategies.

9. **TotalProductCost:**
   - Similar to the standard cost, this represents the total cost for each product in an order.
   - Analyzing this can provide insights into overall costs associated with fulfilling orders.

10. **SalesAmount:**
    - The average sales amount is $486.09, matching the average unit price.
    - This reaffirms the understanding that sales amount is influenced by unit price.

11. **TaxAmt:**
    - The average tax amount is $38.89.
    - This information is crucial for compliance and financial planning.

12. **Freight:**
    - The average freight cost is $12.15.
    - This cost is associated with shipping and logistics, impacting overall expenses.

13. **Other Columns:**
    - The summary provides statistical measures for various categorical and numerical columns, offering insights into customer demographics, education, and order details.

In conclusion, this sales data summary provides a comprehensive view of the orders, their quantities, prices, costs, and associated financial aspects. Analyzing this information can guide strategic decision-making, such as pricing adjustments, inventory management, and the optimization of discount strategies.

In [ ]:
joined_df.summary().show()

# VISUALIZAION
We created a barplot of the count of the content of the following columns  'EnglishProductName', 'MaritalStatus', 'Gender', 'TotalChildren', 'NumberChildrenAtHome',    'CommuteDistance', 'EnglishEducation', 'HouseOwnerFlag', 'NumberCarsOwned',     'EnglishPromotionName', 'CurrencyName', 'SalesTerritoryRegion', 'SalesTerritoryCountry',     'SalesTerritoryGroup', 'StateProvinceName', 'EnglishCountryRegionName', 'City',     'OrderYear', 'OrderMonth', 'DayOfWeek'

 `EnglishProductName`
- **Top Products:**
  - The product "Water Bottle - 30 oz." has the highest count, indicating it is a popular item with 4244 sales.
  - Other popular items include "Patch Kit/8 Patches," "Mountain Tire Tube," and "Road Tire Tube."

- **Helmet Sales:**
  - Helmets, such as "Sport-100 Helmet, Black" and "Sport-100 Helmet, Red," are also among the top-selling products.
  - This suggests a significant demand for safety gear, which could be influenced by the type of customers or specific promotions.

- **Tire Tubes:**
  - Both "Mountain Tire Tube" and "Road Tire Tube" are well-represented in the top products, indicating a demand for bicycle maintenance accessories.

- **Bottle Cages:**
  - "Mountain Bottle Cage" and "Road Bottle Cage" are popular, reflecting interest in accessories for bike customization.

- **Tire Varieties:**
  - Different tire types, including "HL Mountain Tire," "ML Mountain Tire," "LL Road Tire," and others, demonstrate diversity in the product offering.

- **Promotional Products:**
  - Products like "AWC Logo Cap" and "Bike Wash - Dissolver" also have a substantial count, suggesting they may be associated with promotions or branding efforts.



- **Marital Status:**
  - The majority of customers are married (M), comprising 55% of the dataset, indicating a significant presence of families within the customer base. Single customers (S) constitute the remaining 45%.

- **Gender:**
  - The gender distribution is almost evenly split, with 50.3% male (M) customers and 49.7% female (F) customers. This balance suggests a diverse customer demographic.

- **Total Children:**
  - The distribution of the total number of children showcases a range of family sizes. The dataset contains customers with 0 to 5 children, with a concentration around 0 and 2 children.

- **Number of Children at Home:**
  - The majority of customers have no children at home, while the count decreases as the number of children at home increases. This pattern aligns with expectations, as households with children tend to be less common.

- **Commute Distance:**
  - The analysis of commute distances reveals that a substantial number of customers have short commutes (0-1 miles). This information could be crucial for targeting local marketing efforts.

- **Education Level:**
  - The educational background of customers varies, with the most common levels being Bachelors and Partial College. This diversity suggests a broad range of consumer demographics.

- **House Owner Flag:**
  - More than two-thirds of customers own houses, as indicated by HouseOwnerFlag = 1. This insight can be valuable for businesses catering to homeowners.

- **Number of Cars Owned:**
  - The distribution of the number of cars owned reflects a diverse customer base. A significant portion owns 2 cars, while a sizable group has 0 or 1 car.

- **Promotion Name:**
  - The majority of transactions (97%) occur with no associated discounts. However, there are instances of volume discounts and specific promotions, highlighting the potential impact of promotional strategies on sales.

- **Currency Name:**
  - US Dollar is the dominant currency used for transactions, with more than half of the orders denominated in USD. This information is crucial for financial reporting and analysis.

- **Sales Territory Region, Country, and Group:**
  - North America, particularly the United States, emerges as the dominant sales region and country. The majority of sales territory groups belong to North America and Europe, providing insights into geographical sales distribution.

- **State Province Name and Country Region Name:**
  - The count of orders from specific state provinces, such as California and British Columbia, indicates geographic concentration. This can inform targeted marketing and inventory strategies for these regions.

- **City:**
  - London stands out with a significant count, suggesting a notable presence of customers from this city. Further investigation into London-specific trends may reveal valuable insights.

- **Order Year and Month:**
  - Most orders (88%) are from the year 2013, with December exhibiting the highest order count. This aligns with common retail trends, indicating potential holiday-related sales peaks during this month.

- **Day of Week:**
  - The distribution of orders across different days of the week appears relatively consistent, with no significant variations. This information can be valuable for logistical and operational planning.

These intensive insights provide a nuanced understanding of customer demographics, purchasing behavior, and sales dynamics, enabling businesses to tailor their strategies more effectively. If you have specific aspects you'd like to explore further or additional questions, feel free to let me know!


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt


# List of columns to consider
columns_of_interest = [
    'EnglishProductName', 'MaritalStatus', 'Gender', 'TotalChildren', 'NumberChildrenAtHome',
    'CommuteDistance', 'EnglishEducation', 'HouseOwnerFlag', 'NumberCarsOwned',
    'EnglishPromotionName', 'CurrencyName', 'SalesTerritoryRegion', 'SalesTerritoryCountry',
    'SalesTerritoryGroup', 'StateProvinceName', 'EnglishCountryRegionName', 'City',
    'OrderYear', 'OrderMonth', 'DayOfWeek'
]

# Create a count table and bar plot for each column
for column in columns_of_interest:
    # Count table
    count_table = joined_df.groupBy(column).count().orderBy('count', ascending=False)
    
    # Display count table
    print(f"Count Table for {column}:")
    count_table.show()

    # Convert the Spark DataFrame column to a Pandas DataFrame for plotting
    column_data = count_table.toPandas()

    # Plotting
    plt.figure(figsize=(12, 6))
    plt.bar(column_data[column], column_data['count'])
    plt.title(f'Count of {column}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.show()


In [ ]:
joined_df.columns


- **Age:** This column represents the different age groups for which the average unit price is calculated.

- **AverageUnitPrice:** This column shows the calculated average unit price for each corresponding age group. It is the mean value of the 'UnitPrice' column for each age group.

Insights:

1. **High Average Unit Prices:** Age groups 45, 58, 56, 60, and 89 have relatively higher average unit prices, indicating that products purchased by individuals in these age groups tend to be more expensive on average.

2. **Low Average Unit Prices:** Age groups 103, 91, 93, 96, and 87 have lower average unit prices, suggesting that products purchased by individuals in these age groups tend to be less expensive on average.

3. **Varied Spending Patterns:** The table reflects a wide range of average unit prices across different age groups, indicating diverse spending patterns among customers of different ages.

4. **Outliers:** Some age groups, such as 45, have notably high average unit prices, potentially indicating a preference for premium or luxury products in that age group.

5. **Age 104:** There is only one entry for age 104, and it has a relatively high average unit price. However, interpretations for such small sample sizes should be made cautiously.

These insights can be valuable for marketing and sales strategies, helping businesses understand the purchasing behavior of different age groups and tailor their offerings accordingly.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
from pyspark.sql.types import IntegerType


# Calculate the average UnitPrice by Age
average_unit_price_by_age = joined_df.groupBy("Age").agg(avg("UnitPrice").alias("AverageUnitPrice"))

# Cast 'Age' column to IntegerType
average_unit_price_by_age = average_unit_price_by_age.withColumn("Age", average_unit_price_by_age["Age"].cast(IntegerType()))

# Convert PySpark DataFrame to Pandas DataFrame
average_unit_price_by_age_pd = average_unit_price_by_age.toPandas()

# Sort the Pandas DataFrame by AverageUnitPrice in descending order
average_unit_price_by_age_pd = average_unit_price_by_age_pd.sort_values(by='AverageUnitPrice', ascending=False)

# Set the style of seaborn for better visualization
sns.set(style="whitegrid")

# Create a bar plot
plt.figure(figsize=(24, 16))
sns.barplot(x='Age', y='AverageUnitPrice', data=average_unit_price_by_age_pd, palette='viridis')
plt.title('Average UnitPrice by Age (Descending Order)')
plt.xlabel('Age')
plt.ylabel('Average UnitPrice')
plt.show()
# Display the table of the group by 'Age' and average unit price
average_unit_price_by_age.show(300)




- **EnglishProductName:** This column represents the names of different products.

- **AveragePrice:** This column represents the average price of each product.

The table is sorted in descending order based on the 'AveragePrice' column, meaning the products with the highest average price are listed first. Here are a few observations:

1. **Mountain-200 Black, 46:** This product has the highest average price, indicating that it is relatively expensive compared to other products.

2. **Touring-1000 Yellow, 46:** This product also has a high average price, making it one of the more expensive items.

3. **Touring-1000 Blue, 46:** Similar to the yellow variant, the blue version of the Touring-1000 has a high average price.

4. **Short-Sleeve Classic Jersey, M:** This is an example of a product with a lower average price.

5. **Women's Mountain Shorts, M:** Another product with a relatively lower average price.

The table provides insights into the pricing distribution across different product categories, helping you identify products with varying price points.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg


# Calculate the average price by 'EnglishProductName'
average_price_by_product = joined_df.groupBy("EnglishProductName").agg(avg("UnitPrice").alias("AveragePrice"))
average_price_by_product.show(300)
# Convert PySpark DataFrame to Pandas DataFrame
average_price_by_product_pd = average_price_by_product.toPandas()

# Sort the Pandas DataFrame by AveragePrice in descending order
average_price_by_product_pd = average_price_by_product_pd.sort_values(by='AveragePrice', ascending=False)

# Set the style of seaborn for better visualization
sns.set(style="whitegrid")

# Create a bar plot
plt.figure(figsize=(24, 32))
sns.barplot(x='AveragePrice', y='EnglishProductName', data=average_price_by_product_pd, palette='viridis')
plt.title('Average Price by EnglishProductName (Descending Order)')
plt.xlabel('Average Price')
plt.ylabel('EnglishProductName')
plt.show()


Certainly! Let's delve deeper into the insights and draw real-world implications:

1. **MaritalStatus:**
   - Customers with MaritalStatus 'S' might have higher disposable income, as they are not supporting a family. Targeting promotions or products that appeal to independent individuals could be beneficial.

2. **Gender:**
   - The slightly higher average expenditure by female customers could indicate a potential market for premium or personalized products tailored to female preferences.

3. **TotalChildren and NumberChildrenAtHome:**
   - Customers with one or two children seem to spend more. This insight can guide marketing strategies, such as family-oriented promotions or bundles.

4. **CommuteDistance:**
   - Customers with a shorter commute distance might be more willing to spend on convenience. Products tailored to this group, such as home delivery services, could be explored.

5. **EnglishEducation:**
   - Customers with a 'Bachelors' education level may value quality and brand reputation. Marketing efforts can focus on highlighting these aspects to attract similar customers.

6. **HouseOwnerFlag:**
   - Homeowners might be more stable in their living situation, allowing for greater willingness to invest in higher-priced items or home-related products.

7. **NumberCarsOwned:**
   - Customers without cars may allocate more of their budget to other lifestyle categories. Marketing efforts could emphasize products that enhance their non-driving experiences.

8. **EnglishPromotionName:**
   - The success of the 'Touring-1000 Promotion' suggests that exclusive or limited-time promotions can drive higher spending. Similar strategies could be employed for other products.

9. **CurrencyName:**
   - Customers using 'Deutsche Mark' and 'French Franc' might be international travelers or expatriates. Tailoring products or services to this demographic can enhance their experience.

10. **SalesTerritoryRegion and SalesTerritoryCountry:**
    - Regions or countries with higher average unit prices could be targeted for premium product launches or marketing campaigns.

11. **SalesTerritoryGroup:**
    - The 'Pacific' group, with the highest average unit prices, might represent a wealthier market. Customizing offerings to meet the preferences of this group could lead to increased sales.

12. **StateProvinceName:**
    - States with higher average unit prices could be prioritized for marketing campaigns focusing on premium or exclusive products.

13. **OrderMonth and DayOfWeek:**
    - Higher spending in June and on Thursdays suggests potential patterns in consumer behavior. Seasonal promotions or mid-week specials could be tailored to these trends.

Understanding these insights allows businesses to make informed decisions regarding product development, marketing strategies, and customer targeting. By tailoring approaches based on customer characteristics, businesses can enhance the overall customer experience and drive increased sales.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt


# List of columns to consider
columns_of_interest = [
    'EnglishProductName', 'MaritalStatus', 'Gender', 'TotalChildren', 'NumberChildrenAtHome',
    'CommuteDistance', 'EnglishEducation', 'HouseOwnerFlag', 'NumberCarsOwned',
    'EnglishPromotionName', 'CurrencyName', 'SalesTerritoryRegion', 'SalesTerritoryCountry',
    'SalesTerritoryGroup', 'StateProvinceName', 'EnglishCountryRegionName', 'City',
    'OrderYear', 'OrderMonth', 'DayOfWeek'
]

# Create a count table and bar plot for each column
for column in columns_of_interest:
    # Count table
    count_table = joined_df.groupBy(column).count().orderBy('count', ascending=False)
    
    # Display count table
    print(f"Count Table for {column}:")
    count_table.show()

    # Convert the Spark DataFrame column to a Pandas DataFrame for plotting
    column_data = count_table.toPandas()

    # Plotting
    plt.figure(figsize=(12, 6))
    plt.bar(column_data[column], column_data['count'])
    plt.title(f'Count of {column}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg



# List of columns for which you want to create bar plots
columns_to_plot = [
    'EnglishProductName', 'MaritalStatus', 'Gender', 'TotalChildren', 'NumberChildrenAtHome',
    'CommuteDistance', 'EnglishEducation', 'HouseOwnerFlag', 'NumberCarsOwned',
    'EnglishPromotionName', 'CurrencyName', 'SalesTerritoryRegion', 'SalesTerritoryCountry',
    'SalesTerritoryGroup', 'StateProvinceName', 'EnglishCountryRegionName', 'City',
    'OrderYear', 'OrderMonth', 'DayOfWeek'
]

# Create subplots
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(20, 20))
fig.subplots_adjust(hspace=0.5)

# Flatten the axes for easier iteration
axes = axes.flatten()

# Loop through columns and create bar plots
for i, column in enumerate(columns_to_plot):
    # Calculate the average UnitPrice for each unique value in the column
    average_price_by_column = joined_df.groupBy(column).agg(avg("UnitPrice").alias("AverageUnitPrice"))

    # Convert PySpark DataFrame to Pandas DataFrame
    average_price_by_column_pd = average_price_by_column.toPandas()

    # Sort the Pandas DataFrame by AverageUnitPrice in descending order
    average_price_by_column_pd = average_price_by_column_pd.sort_values(by='AverageUnitPrice', ascending=False)

    # Create a bar plot
    sns.barplot(ax=axes[i], x='AverageUnitPrice', y=column, data=average_price_by_column_pd, palette='viridis')
    axes[i].set_title(f'Average UnitPrice by {column}')
    axes[i].set_xlabel('Average UnitPrice')
    axes[i].set_ylabel(column)

# Show the plots
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg



# Columns to plot against the average UnitPrice
columns_to_plot = [
    'EnglishProductName', 'MaritalStatus', 'Gender', 'TotalChildren', 'NumberChildrenAtHome',
    'CommuteDistance', 'EnglishEducation', 'HouseOwnerFlag', 'NumberCarsOwned',
    'EnglishPromotionName', 'CurrencyName', 'SalesTerritoryRegion', 'SalesTerritoryCountry',
    'SalesTerritoryGroup', 'StateProvinceName', 'EnglishCountryRegionName', 'City',
    'OrderYear', 'OrderMonth', 'DayOfWeek'
]

for column_to_plot in columns_to_plot:
    # Calculate the average UnitPrice for each unique value in the column
    average_price_by_column = joined_df.groupBy(column_to_plot).agg(avg("UnitPrice").alias("AverageUnitPrice"))

    # Convert PySpark DataFrame to Pandas DataFrame
    average_price_by_column_pd = average_price_by_column.toPandas()

    # Sort the Pandas DataFrame by AverageUnitPrice in descending order
    average_price_by_column_pd = average_price_by_column_pd.sort_values(by='AverageUnitPrice', ascending=False)

    # Print the table for each plot
    print(f"Table for {column_to_plot}:")
    print(average_price_by_column_pd)

    # Set the style of seaborn for better visualization
    sns.set(style="whitegrid")

    # Create a bar plot
    plt.figure(figsize=(24, 16))
    sns.barplot(x='AverageUnitPrice', y=column_to_plot, data=average_price_by_column_pd, palette='viridis')
    plt.title(f'Average UnitPrice by {column_to_plot}')
    plt.xlabel('Average UnitPrice')
    plt.ylabel(column_to_plot)
    plt.show()


# model 

##  LOGISTICS REGRESSION

### OBJECTIVES 

The main objective of my logistic regression model is to predict the probability that a given observation belongs to a particular category or class. In this case, I'm using the "Gender_index" as the target variable, and my goal is to predict the probability of each observation being classified as a specific gender.

To achieve this, I start by indexing the "Gender_index" column to convert categorical gender labels into numerical values. This step is necessary as logistic regression typically requires numerical input.

Next, I assemble the feature columns, excluding "Gender_index," into a vector. This vector serves as the input for my logistic regression model.

I then train the logistic regression model using the assembled features and the indexed gender labels. The model learns to predict the probability of an observation belonging to a particular gender.

To streamline the entire process, I've encapsulated it in a pipeline, which includes string indexing, vector assembling, and model training. This makes the workflow more manageable and reproducible.

To evaluate the performance of my model, I use the Area Under the ROC Curve (AUC-ROC), a common metric for binary classification models. This metric assesses the trade-off between the true positive rate and false positive rate, providing insights into the model's effectiveness.

In summary, my logistic regression model aims to provide a probabilistic prediction of gender based on the input features, facilitating the binary classification of observations into specific gender categories.


1. **Area Under ROC Curve (AUC):**
   - **Value:** 0.5434
   - **Explanation:** The AUC is a metric that quantifies the ability of a binary classification model to distinguish between the positive and negative classes. A value of 0.5 suggests no discrimination (random guessing), and a value of 1.0 indicates perfect discrimination. Your AUC of 0.5434 suggests a model that has some ability to discriminate, but the performance may not be very strong.

2. **Coefficients:**
   - **Values:** An array of numerical coefficients for each feature.
   - **Explanation:** These coefficients represent the weights assigned to each feature by the logistic regression model. Positive coefficients indicate a positive correlation with the log-odds of the positive class, while negative coefficients indicate a negative correlation. Features with larger absolute values have a stronger impact on the prediction.

3. **Intercept:**
   - **Value:** -50.96727297970031
   - **Explanation:** The intercept represents the log-odds of the positive class when all features are zero. In logistic regression, the logistic function is applied to the linear combination of features and coefficients, and the intercept is added. The result is transformed into probabilities.

In summary, your model has been trained to predict the target variable `Gender_index`. The AUC suggests modest discriminative ability, and the coefficients provide insights into how each feature contributes to the prediction. If you have specific questions about certain coefficients or features, feel free to ask!

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Assuming 'joined_df_transformed' is your DataFrame
# If 'Gender_index' is not numeric, you need to index it first
gender_indexer = StringIndexer(inputCol="Gender_index", outputCol="label")

# Exclude the 'Gender_index' column from the feature columns
feature_cols = [col for col in joined_df_transformed.columns if col != 'Gender_index']

# Vector Assembler to assemble feature columns into a vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Create a pipeline
pipeline = Pipeline(stages=[gender_indexer, assembler, lr])

# Split the data into training and testing sets
(training_data, testing_data) = joined_df_transformed.randomSplit([0.8, 0.2], seed=42)

# Train the model
model = pipeline.fit(training_data)

# Make predictions on the testing data
predictions = model.transform(testing_data)

# Evaluate the model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label")
area_under_curve = evaluator.evaluate(predictions)

# Print the Area Under ROC Curve
print("Area Under ROC Curve: {:.4f}".format(area_under_curve))

# Display the model's coefficients
print("Coefficients: " + str(model.stages[-1].coefficients))

# Display the model's intercept
print("Intercept: " + str(model.stages[-1].intercept))


In [ ]:
import pandas as pd
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Convert PySpark DataFrame to Pandas DataFrame
predictions_pd = predictions.select(['probability', 'label']).toPandas()

# Extract the probability of the positive class (1)
y_prob = predictions_pd['probability'].apply(lambda x: x[1])

# Calculate ROC curve
fpr, tpr, _ = roc_curve(predictions_pd['label'], y_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()



In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml import Pipeline

# Assuming 'joined_df_transformed' is your DataFrame
# If the target column is not numeric, you need to index it first
label_indexer = StringIndexer(inputCol="EnglishProductName_index", outputCol="label")

# Exclude the target column from the feature columns
feature_cols = [col for col in joined_df_transformed.columns if col != 'EnglishProductName_index']

# Vector Assembler to assemble feature columns into a vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Logistic Regression model for multinomial classification
lr = LogisticRegression(featuresCol="features", labelCol="label", family="multinomial")

# Create a pipeline
pipeline = Pipeline(stages=[label_indexer, assembler, lr])

# Split the data into training and testing sets
(training_data, testing_data) = joined_df_transformed.randomSplit([0.7, 0.3], seed=42)

# Train the model
model = pipeline.fit(training_data)

# Make predictions on the testing data
predictions = model.transform(testing_data)

# Evaluate the model
# (Use appropriate evaluation metrics for multinomial classification)


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

# Print the accuracy
print("Accuracy: {:.2%}".format(accuracy))

# Confusion Matrix
conf_matrix = predictions.groupBy("label", "prediction").count()
print("Confusion Matrix:")
conf_matrix.show()

# Other metrics
precision = evaluator.setMetricName("weightedPrecision").evaluate(predictions)
recall = evaluator.setMetricName("weightedRecall").evaluate(predictions)
f1_score = evaluator.setMetricName("f1").evaluate(predictions)

print("Precision: {:.2%}".format(precision))
print("Recall: {:.2%}".format(recall))
print("F1 Score: {:.2%}".format(f1_score))


## KNN

The main objective of the k-Nearest Neighbors (k-NN) model that I've implemented is to predict the 'Gender_index' variable based on the features present in the dataset. In this context, I used the 'joined_df_transformed' DataFrame, assuming it contains the necessary features and the target variable. The process involves setting up a pipeline that includes a StringIndexer (to handle non-numeric values in the 'Gender_index'), a VectorAssembler to bring together the feature columns, and the k-NN classifier for the actual prediction task.

After setting up the pipeline, the model is trained using the provided training data. Subsequently, predictions are generated for the testing data, allowing for an evaluation of the model's accuracy. The MulticlassClassificationEvaluator is employed to assess how well the k-NN model performs on unseen data, providing a quantitative measure of its predictive capabilities. This accuracy metric is crucial in determining the reliability and effectiveness of the k-NN model in classifying gender indices based on the given dataset features.


## model explaintion 


The k-Nearest Neighbors (k-NN) model has been evaluated with the provided testing data, and the results are summarized in the classification report and confusion matrix. The classification report displays various metrics such as precision, recall, and F1-score for each class (0.0 and 1.0) along with the overall accuracy. In this case, the model achieved an accuracy of approximately 52%, with balanced precision, recall, and F1-scores for both classes. The confusion matrix further details the number of true positive, true negative, false positive, and false negative instances, showing how well the model performed in classifying instances belonging to each class. The k-NN model's performance indicates a moderate level of effectiveness in predicting the 'Gender_index' variable based on the given features.

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Assuming 'joined_df_transformed' is your DataFrame
# If 'Gender_index' is not numeric, you need to index it first
gender_indexer = StringIndexer(inputCol="Gender_index", outputCol="label")

# Exclude the 'Gender_index' column from the feature columns
feature_cols = [col for col in joined_df_transformed.columns if col != 'Gender_index']

# Vector Assembler to assemble feature columns into a vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Transform PySpark DataFrame to Pandas DataFrame
pandas_df = joined_df_transformed.toPandas()

# Split the data into training and testing sets (using Pandas DataFrame)
train_size = int(0.8 * len(pandas_df))
train_data, test_data = pandas_df.iloc[:train_size, :], pandas_df.iloc[train_size:, :]

# Convert Pandas DataFrame back to PySpark DataFrame
train_data_spark = spark.createDataFrame(train_data)
test_data_spark = spark.createDataFrame(test_data)

# Train k-NN model using scikit-learn
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_data[feature_cols], train_data['Gender_index'])

# Make predictions on the testing data
predictions = knn.predict(test_data[feature_cols])

# Evaluate the model
print("Classification Report:\n", classification_report(test_data['Gender_index'], predictions))
print("Confusion Matrix:\n", confusion_matrix(test_data['Gender_index'], predictions))


In [ ]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split

# Assuming 'joined_df_transformed' is your DataFrame
# If 'Gender_index' is not numeric, you need to index it first
gender_indexer = StringIndexer(inputCol="Gender_index", outputCol="label")

# Exclude the 'Gender_index' column from the feature columns
feature_cols = [col for col in joined_df_transformed.columns if col != 'Gender_index']

# Vector Assembler to assemble feature columns into a vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Transform PySpark DataFrame to Pandas DataFrame
pandas_df = joined_df_transformed.toPandas()

# Split the data into training and testing sets (using Pandas DataFrame)
train_data, test_data = train_test_split(pandas_df, test_size=0.2, random_state=42)

# Convert Pandas DataFrame back to PySpark DataFrame
train_data_spark = spark.createDataFrame(train_data)
test_data_spark = spark.createDataFrame(test_data)

# Train k-NN model using scikit-learn
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_data[feature_cols], train_data['Gender_index'])

# Make predictions on the testing data
predictions = knn.predict(test_data[feature_cols])

# Evaluate the model
print("Classification Report:\n", classification_report(test_data['Gender_index'], predictions))
print("Confusion Matrix:\n", confusion_matrix(test_data['Gender_index'], predictions))

# Visualization
plt.figure(figsize=(10, 6))

# Plot training points
plt.scatter(train_data[feature_cols[0]], train_data[feature_cols[1]], c=train_data['Gender_index'], cmap='viridis', label='Training Data', edgecolors='k', s=50)

# Plot testing points
plt.scatter(test_data[feature_cols[0]], test_data[feature_cols[1]], c=predictions, cmap='viridis', marker='X', label='Predictions', edgecolors='k', s=100)

plt.xlabel(feature_cols[0])
plt.ylabel(feature_cols[1])
plt.title('k-NN Classification Visualization')
plt.legend()
plt.show()


# CONCLUSION 

Based on the evaluation results of the k-Nearest Neighbors (k-NN) model, it can be concluded that the model exhibits a moderate level of performance in predicting the 'Gender_index' variable. The overall accuracy of approximately 52% suggests that the model's ability to correctly classify instances is marginally better than random chance. However, the balanced precision, recall, and F1-scores for both classes (0.0 and 1.0) indicate that the model shows consistency in performance across different metrics.

The classification report and confusion matrix provide a detailed breakdown of the model's performance, showing that it is not biased toward any specific class. The model's ability to correctly identify instances belonging to both classes is reflected in the balanced nature of the evaluation metrics. While the model's performance is not exceptionally high, it still demonstrates a reasonable capability to make predictions based on the provided features.

It's important to note that the effectiveness of the model may depend on the specific characteristics of the dataset and the features used for prediction. Further refinement, feature engineering, or consideration of alternative modeling approaches could be explored to potentially enhance the predictive performance.